# Projeto 2

## O que é o Projeto 2?

Neste projeto, você deverá programar um jogador automático para o jogo que está aqui.

Pode ser feito individualmente ou em dupla.

# Observação 

Este código é bastante longo e é possível que eu tenha que enviar algumas correções. Posso fazer vídeos para esclarecer algumas dúvidas também.

## Como serei avaliado?

Você deverá implementar uma entre as seguintes opções.

- Jogador para duelo: um jogador contra um jogador
- Jogador para arena: vários jogadores

O seu conceito para este projeto dependerá dos seguintes fatores:

1) A complexidade e performance da estratégia que você implementou. Tente usar condicionais e laços.

2) O quão bem foram implementadas a sua estratégia.

3) O quão bem você explicou sua estratégia e código.

## O que devo entregar?

1) Este notebook ipynb preenchido

2) Um link para vídeo de no máximo 5 minutos (que deve ser incluso aqui) onde você explica a sua estratégia e o seu código. Se for dupla, cada membro deve fazer seu próprio vídeo.

> Sugiro que você use o Loom (https://www.loom.com) para gravar seu vídeo explicando o notebook. É essa a ferramenta que uso para gravar meus vídeos. 

# Nome e RA

## Nome: Brunno Toscano Messias
## RA: 11202020125

## Nome: Marco Antonio Lopes Rodrigues
## RA: 11202020422 

# Vídeo
## Link Marco: https://www.loom.com/share/e29ce582f65945948ebe0e54a4124536
## Link Brunno: https://www.loom.com/share/4943fb036fd24dd984fad35bb3e94e5f

In [ ]:
import random as rd
import time
import numpy as np
import matplotlib.pyplot as plt
import copy
from matplotlib import cm
%matplotlib inline

!pip install celluloid
from celluloid import Camera

# Características do jogo

Aqui apresentamos algumas características do jogo. Mas, para detalhes, você pode ter que descobrir lendo o código.

* **Multiplayer:** Vários jogadores podem jogar simultaneamente. 
* **Baseado em turnos:** Em cada turno, cada jogador recebe o status dos outros jogadores e deve decidir qual ação tomar.

* **Ocorre em um mapa 2D:** Os jogadores se movimentam por um mapa.

In [ ]:
# Configuracoes
max_turnos = 20 # quantidade maxima de turnos a serem realizados
raio = 20 # raio padrao de ataque
step = 10 # tamanho de um passo de movimento
mapa_x = 100 # tamanho do mapa na coordenada x 
mapa_y = 100 # tamanho do mapa na coordenada y
dano_base = 10 # base para dano de um ataque
vida_base = 100 # quantidade de vida padrão

Você deve programar o status inicial de seu jogador.
Isso deve ser feito através de uma função.

Aqui estão os parâmetros que você deve definir:

## Tipo

O seu jogador terá um tipo de elemento. Os possíveis elementos são:
"FOGO", "AGUA", "PLANTA", "LUZ", "ESCURIDAO", "NORMAL"

* Cada jogador pode fazer um ataque especial que corresponde a seu elemento ou um ataque normal.

* Ataques de FOGO são fortes contra PLANTA e fracos contra AGUA.

* Ataques de AGUA são fortes contra FOGO e fracos contra PLANTA.

* Ataques de PLANTA são fortes contra AGUA e fracos contra FOGO.

* Ataques de LUZ são fortes contra ESCURIDAO.

* Ataques de ESCURIDAO são fortes contra LUZ.

* Ataques do tipo NORMAL não são nem fortes e nem fracos. Podem ser realizados por um jogador de qualquer tipo, Mas, se o jogador for do tipo NORMAL, o ataque é mais forte.

O quão forte/fracos são os ataques podem ser vistos no código do jogo.


## Distribuição de pontos
Você deve distribuir **100** pontos inteiros não-negativos. A soma deve ser exatamente $100$. Caso contrário, seu jogo pode não rodar.

Os pontos devem ser distribuídos dentre:
* Vida
* Ataque
* Ataque especial
* Defesa
* Sorte

Você mesmo deve checar como esses parâmetros são usados no jogo.

# Posição 
Escolher uma posição `(x,y)` para seu jogador com a 
seguinte regra:

``step <= x <= mapa_x - step``

E

``step <= y <= mapa_y - step``

## O que a função deve retornar?

Uma lista com os parâmetros na seguinte ordem:

``[vida, ataque, ataque_especial, defesa, sorte, tipo, x, y]``

Veja um exemplo de função para a configuração inicial:


In [ ]:
def inicial_exemplo ():
  vida = 20
  ataque = 20
  ataque_especial = 20
  defesa = 20
  sorte = 20
  tipo = "NORMAL"
  x = 50
  y = 50
  perfil = [vida, ataque, ataque_especial, defesa, sorte, tipo,x,y]
  return perfil

Você pode checar se a sua função é válida com a seguinte função:

In [ ]:
def valida_inicial(func_inic):
  [vida, ataque, ataque_especial, defesa, sorte, tipo,x,y] = func_inic()
  valido = True
  if vida <0 or ataque<0 or ataque_especial<0 or defesa<0 or sorte<0:
    print("Inválido. Algum parâmetro é negativo.")
    valido = False
  if vida+ataque+ataque_especial+defesa+sorte != 100:
    print("Inválido. Soma de parâmetros não é 100.")
    valido = False
  if tipo not in ["FOGO", "AGUA", "PLANTA", "LUZ", "ESCURIDAO", "NORMAL"]:
    print("Inválido. Tipo inválido.")
    valido = False
  if x<step or x>mapa_x-step or y<step or y>mapa_y-step:
    print("Inválido. Posição inválida.")
    valido = False
  return valido

Para a minha função de exemplo, é assim que pode ser checada. 

In [ ]:
valida_inicial(inicial_exemplo)

# Turno

Em cada turno, você pode escolher um movimento da seguinte lista:

"AT": Ataque. Ataca todos os outros jogadores de acordo com a posição deles e seu raio de ataque (e um fator de sorte)

"AE": Ataque Especial. Ataca com seu tipo todos os outros jogadores de acordo com a posição deles e seu raio de ataque (e um fator de sorte)

"DF_INC": Aumenta o parâmetro de defesa

"SORTE_INC": Aumenta o parâmetro de sorte

"N": Move em direção ao Norte.

"S" : Move em direção ao Sul.

"L": Move em direção ao Leste.

"O": Move em direção ao Oeste.

"NO": Move em direção ao Noroeste.

"NL": Move em direção ao Nordeste.

"SL": Move em direção ao Sudeste.

"SO": Move em direção ao Sudoeste.

"FOGO": Muda seu tipo para FOGO.

"AGUA": Muda seu tipo para AGUA.

"PLANTA": Muda seu tipo para PLANTA.

"LUZ": Muda seu tipo para LUZ.

"ESCURIDAO": Muda seu tipo para ESCURIDAO.

"NORMAL": Muda seu tipo para NORMAL.

O movimento decidido pelo turno deve ser programado em uma função. Veja dois exemplos:

In [ ]:
moves = ["AT","AE", "DF_INC", "SORTE_INC", 
         "N","S","L","O","NO","NL","SL","SO",
         "FOGO", "AGUA", "PLANTA", "LUZ", "ESCURIDAO", "NORMAL"]

def turno_aleatorio(status, player): #escolhe um movimento qualquer
  return rd.choice(moves)

def turno_so_ataca(status, player):
  escolhas = ["AT","AE"]
  return rd.choice(escolhas)

Os exemplos acima são bem ingênuos, pois não dependem dos status de outros jogadores.

O que são os parâmetros recebidos pelo turno?

* player: é o **seu** identificador numérico

* status: contém o status de cada jogador!

Vamos ver o código do jogo para ver como os status dos jogadores são guardados.

> Abaixo estão as partes em que você deve colocar seu código!

# Sua estratégia para duelo

Fizemos um jogador com um comportamento agressivo (como de um caçador ou assassino). 

A distribuição de pontos foi baseada nesse comportamento, com enfoque em ataque, e um pouco de sorte e defesa.

1) O jogador irá checar o elemento do inimigo no começo de todo turno, se o elemento for diferente de "NORMAL", o jogador irá trocar para o elemento mais favorável (ex: jogador troca para AGUA se o inimigo estiver com o elemento FOGO).

2) Se o inimigo estiver fora de seu raio de ataque, o persegue até que esteja numa posição em que o ataque acertará (não é necessário monitorar os limites do mapa já que o jogador para de perseguir quando entra numa distância de 20 unidades do inimigo)

3) Quando está abaixo de 50% da vida, continua perseguindo o inimigo mas ocasionalmente aumenta a defesa ao invés de atacar.

OBS: A posição inicial foi colocada longe de (50,50) para mostrar a capacidade de perseguição do jogador

# Sua estratégia para arena
Preencha aqui com a sua estratégia!

# Modifique o código abaixo para criar a sua função de configuração inicial abaixo:

In [ ]:
def inicial():
  vida = 0
  ataque = 50
  ataque_especial = 0
  defesa = 25
  sorte = 25
  tipo = "NORMAL"
  x = 10
  y = 10
  perfil = [vida, ataque, ataque_especial, defesa, sorte, tipo,x,y]
  return perfil

In [ ]:
def inicialArena():
  vida = 20
  ataque = 20
  ataque_especial = 20
  defesa = 20
  sorte = 20
  tipo = "NORMAL"
  x = 50
  y = 50
  perfil = [vida, ataque, ataque_especial, defesa, sorte, tipo,x,y]
  return perfil

# Rode o código abaixo para ver se ela é válida:

In [ ]:
valida_inicial(inicial)

In [ ]:
valida_inicial(inicialArena)

# Modifique o código abaixo para criar a sua função de turno:

In [ ]:
def turno(status, jogador):
  op = 1
  if jogador == 1:
    op = 0
  
  # Troca o elemento para um favorável:
  while status[op]["tipo"] != "NORMAL":
    if especial(status[jogador]["tipo"], status[op]["tipo"]) == True: # Quebra o laço caso o elemento atual seja favorável (para evitar um loop infinito de troca de elementos)
      break
    elif status[op]["tipo"] == "FOGO":
      return "AGUA"
    elif status[op]["tipo"] == "AGUA":
      return "PLANTA"
    elif status[op]["tipo"] == "PLANTA":
      return "FOGO"
    elif status[op]["tipo"] == "LUZ":
      return "ESCURIDAO"
    elif status[op]["tipo"] == "ESCURIDAO":
      return "LUZ"
  
  # Ataca se estiver dentro do raio de ataque padrão e com 50% ou mais de vida (mesmo que o elemento do jogador seja NORMAL, o ataque especial causará a mesma quantidade de dano):
  if prox(status, jogador, op, raio, mapa_x, mapa_y) == True and status[jogador]["vida"] >= 50:
      return "AE"
  
  # Age conforme a parte da última condição que não foi cumprida:
  else:
    if prox(status, jogador, op, raio, mapa_x, mapa_y) == False: # Se estiver fora do raio de ataque, persegue o inimigo.
      dir_x = ""
      dir_y = ""
      if status[op]["y"] > status[jogador]["y"]:
        dir_y = "N"
      elif status[op]["y"] < status[jogador]["y"]:
        dir_y = "S"
      if status[op]["x"] > status[jogador]["x"]:
        dir_x = "L"
      elif status[op]["x"] < status[jogador]["x"]:
        dir_x = "O"
      return dir_y+dir_x
    else: # Se estiver dentro do raio, a condição não cumprida foi a de vida, então aumenta defesa ou ataca.
        return rd.choice(["AE", "DF_INC"])

In [ ]:
def turnoArena(status, jogador):
  # Metapod
  return "AE"

# Código do jogo

Você não precisa ler e nem entender o código todo, mas procure ver como os parâmetros são usados para decidir sua estratégia.

In [ ]:
# Calcula a nova posicao de um jogador
# Parametros x,y: posicao atual
# dir: direcao escolhida
# Retorna (x_novo, y_novo) com a nova posicao
def pos(x,y,dir):
  if dir == "N":
    x_novo = x
    y_novo = y+step
  elif dir == "S":
    x_novo = x
    y_novo = y-step
  elif dir == "L":
    x_novo = x+step
    y_novo = y
  elif dir == "O":
    x_novo = x-step
    y_novo = y
  elif dir == "NO":
    x_novo = x-step
    y_novo = y+step
  elif dir == "NL":
    x_novo = x+step
    y_novo = y+step
  elif dir == "SO":
    x_novo = x-step
    y_novo = y-step
  elif dir == "SL":
    x_novo = x+step
    y_novo = y-step

  # verificando e ajustando para posicao válida
  if y_novo < step:
    y_novo = step
  elif y_novo >= mapa_y-step:
    y_novo = mapa_y-step
  if x_novo < step:
    x_novo = step
  elif x_novo >= mapa_x-step:
    x_novo = mapa_x-step
    
  return (x_novo,y_novo)

Exemplo de uso:

In [ ]:
pos(30,50,"N")

In [ ]:
# Retorna: True se tipo1 é forte contra o tipo2
# False se o tipo1 é fraco contra o tipo2
# 0 se não é nem fraco e nem forte
def especial(tipo1, tipo2):
  if tipo1=="AGUA" and tipo2=="FOGO":
    return True
  elif tipo1=="FOGO" and tipo2=="PLANTA":
    return True
  elif tipo1=="PLANTA" and tipo2=="AGUA":
    return True
  elif (tipo1=="LUZ" and tipo2=="ESCURIDAO") or (tipo1=="ESCURIDAO" and tipo2=="LUZ"):
    return True
  if tipo2=="AGUA" and tipo1=="FOGO":
    return False
  elif tipo2=="FOGO" and tipo1=="PLANTA":
    return False
  elif tipo2=="PLANTA" and tipo1=="AGUA":
    return False
  return 0

Exemplo de uso:

In [ ]:
especial("AGUA","FOGO")

In [ ]:
# Calcula o status inicial dada uma funcao inicial
# Parâmetro: uma função inicial
# Retorna: um dicionário com o status inicial do jogador
def calcula_status_individual_inicial(funcao_inicial):
  perfil = funcao_inicial()
  vida = vida_base*(1+perfil[0]/100)
  ataque = (1+perfil[1]/100)
  ataque_especial = (1+perfil[2]/100)
  defesa = (1+perfil[3]/100)
  sorte = (1+perfil[4]/100)
  status_individual = dict()
  status_individual["vida"] = int(vida)
  status_individual["ataque"] = ataque
  status_individual["ataque_esp"] = ataque_especial
  status_individual["defesa"] = defesa
  status_individual["sorte"] = sorte
  status_individual["tipo"] = perfil[5]
  status_individual["x"] = perfil[6]//step*step
  status_individual["y"] = perfil[7]//step*step
  return status_individual

Exemplo de uso:

In [ ]:
status_individual = calcula_status_individual_inicial(inicial_exemplo)

o ``status_individual`` retornado é um **dictionary**. Para acessar o valor de cada parâmetro, digitamos o nome do parâmetro:

In [ ]:
status_individual["vida"]

In [ ]:
status_individual["defesa"]

Mas temos vários jogadores!
Então temos uma função que calcula o status de cada jogador:

In [ ]:
 # Calcula o status iniciais de todos os jogadores
 # Recebe: iniciais = uma lista de funções iniciais
 # Retorna: um dictionary de status iniciais
 def calcula_status_inicial(status, iniciais):
  msg = "" # Mensagem do que aconteceu
  n = len(iniciais)
  for player in range(0,n):
    if valida_inicial(iniciais[player]):
      status[player] = calcula_status_individual_inicial(iniciais[player])
      msg = msg+"Jogador "+str(player)+" com o seguinte status:\n"
      msg = msg+"Vida: "+str(status[player]["vida"])+"\n"
      msg = msg+"Posicao: ("+str(status[player]["x"])+","+str(status[player]["y"])+")\n"
      msg = msg+"Tipo: "+str(status[player]["tipo"])+"\n"
      msg = msg+"Ataque: "+str(status[player]["ataque"])+"\n"
      msg = msg+"Ataque Especial: "+str(status[player]["ataque_esp"])+"\n"
      msg = msg+"Defesa: "+str(status[player]["defesa"])+"\n"
      msg = msg+"Sorte: "+str(status[player]["sorte"])+"\n\n"
    else:
      msg = "Jogador "+str(player)+" com configuração inválida\n"
      return (msg,False)
  return (msg, True)

Exemplo de uso com 3 jogadores:

In [ ]:
iniciais = [inicial_exemplo,inicial_exemplo,inicial_exemplo]
status = dict()
(msg,cond) = calcula_status_inicial(status, iniciais)
print(cond, "todos os jogadores tem configuração inicial válida")
print(msg)

Veja como acessar o status do jogador 0:

In [ ]:
status[0]

Para acessar o valor do parâmetro de 'ataque' do jogador 0:

In [ ]:
status[0]['ataque']

In [ ]:
for jogador in status.keys():
  print(jogador)
  print(status[jogador])
  print("Jogador",jogador,"tem vida",status[jogador]["vida"])

## Importante! 

Os status dos jogadores será utilizado no jogo inteiro!

Veja, por exemplo, a função que decide se um jogador p1 e um jogador p2 estão a uma distância $\leq r$ (que tipo de distância é esta?).

In [ ]:
def prox(status, p1, p2, r, mapa_x, mapa_y):
  # posicoes dos jogadores p1 e p2
  x1 = status[p1]["x"]
  y1 = status[p1]["y"]
  x2 = status[p2]["x"]
  y2 = status[p2]["y"]

  dx = abs(x1-x2)
  dy = abs(y1-y2)

  if dx<=r and dy<=r:
    return True
  return False

Aqui está a função que decide se existe no máximo um jogador vivo (e o jogo deve acabar).

## Importante! ``status.keys()`` é uma lista com os identificadores numéricos todos os jogadores

In [ ]:
def acabou(status):
  c = 0 #vai contar os numero de jogadores vivos
  for player in status.keys():
    if status[player]["vida"]>0: 
      c = c+1
  if c <= 1: #acabou!
    return True 
  else:
    return False

A função abaixo é usada para atualizar tipo, defesa e sorte:

In [ ]:
def atualiza_tipo_defesa_sorte(status, player, move):
  tipos = ["NORMAL","AGUA","FOGO","PLANTA", "LUZ", "ESCURIDAO"]
  msg = "Jogador "+str(player)+" escolheu movimento inválido "+str(move)+"\n"
  if move=="DF_INC":
    increase = rd.randint(5,10)
    status[player]["defesa"] = (1+increase/100)*status[player]["defesa"]
    msg = "Jogador "+str(player)+" aumentou sua defesa em "+str(increase)+"%\n"
  elif move=="SORTE_INC":
    increase = rd.randint(5,10)
    status[player]["sorte"] = (1+increase/100)*status[player]["sorte"]
    msg = "Jogador "+str(player)+" aumentou sua sorte em "+str(increase)+"%\n"
  elif move in tipos:
        status[player]["tipo"] = move
        msg = "Jogador "+str(player)+" mudou seu tipo para "+str(move)+"\n"
  return msg

A função a seguir é usada para atualizar as posições.

``move_list`` contém as informações da movimentações escolhidas

Esta função lida com os casos em que os jogadores colidem.

In [ ]:
def atualiza_posicoes(status, move_list):
  msg = ""
  # realiza movimentacoes
  for (player, move) in move_list:
    (x,y) = pos(status[player]["x"],status[player]["y"],move)
    msg = msg+"Jogador "+str(player)+" se move para ("+str(x)+","+str(y)+")\n"
    status[player]["x"] = x
    status[player]["y"] = y

# verifica e lida com colisoes
  mapa = dict()
  for x in range(0,mapa_x, step):
    mapa[x] = dict()
    for y in range(0,mapa_y, step):
      mapa[x][y] = []
  for player in status.keys():
    mapa[status[player]["x"]][status[player]["y"]].append(player)

  colisao = True
  while colisao:
    colisao = False
    for x in range(0,mapa_x, step):
      for y in range(0,mapa_y, step):
        if len(mapa[x][y])>1: # tem mais de um jogador em (x,y)!
          msg = msg+"Colisão em ("+str(x)+","+str(y)+"):\n Jogadores"
          for player in mapa[x][y]:
            msg = msg+" "+str(player)
          msg = msg+"\n"
          colisao = True
          l = mapa[x][y].copy()
          for player in l:
            mapa[x][y].remove(player)
            x_novo = rd.randint(step,mapa_x-step)//step*step #para garantir que a posicao é multiplo de step
            y_novo = rd.randint(step,mapa_y-step)//step*step
            status[player]["x"] = x_novo
            status[player]["y"] = y_novo
            msg = msg+"Jogador "+str(player)+" reaparece em ("+str(x_novo)+","+str(y_novo)+")\n"
            mapa[x_novo][y_novo].append(player)

  return msg

A função a seguir é utilizada para realizar os ataques!

``at_list`` é a lista de ataques a serem realizados no turno

In [ ]:
def atualiza_ataques(status, at_list):
  n_attackers = len(at_list)
  eps = 0.00001 #para evitar problemas numéricos
  msg = ""
  at_list_log = []
  for (attacker, attack) in at_list:
    #raio de ataque
    r = int(raio*status[attacker]["ataque"]*status[attacker]["sorte"]*(0.8+rd.random()/2.5))

    # at_list_log é usada para guardar um log dos ataques realizados
    # não influencia no ataque
    at_list_log.append([attacker, attack, r, status[attacker]["tipo"]])

    if attack == "AT": #ataque normal
      msg += "Jogador "+str(attacker)+" ataca! Seu raio de ataque é "+str(r)+"\n"
    else: #ataque de elemento/especial
      msg += "Jogador "+str(attacker)+" ataca com seu elemento! Seu raio de ataque é "+str(r)+"\n"

    for player in status.keys(): # ver quem foi acertado pelo ataque
      if player != attacker and prox(status, player, attacker, r, mapa_x, mapa_y) and status[player]["vida"] > 0:
        # multiplicadores que vão determinar o dano
        multi1 = status[attacker]["ataque"]
        multi2 = max(1/status[player]["defesa"],0.2)
        multi3 = 1
        if status[attacker]["tipo"]=="NORMAL":
          multi3 = 1.2
        elif attack=="AE" and especial(status[attacker]["tipo"], status[player]["tipo"])==True:
          multi3 = 1.5
        elif  attack=="AE" and especial(status[attacker]["tipo"], status[player]["tipo"])==False:
          multi3 = 0.7
        dano = int(dano_base*multi1*multi2*multi3)
        status[player]["vida"] = max(status[player]["vida"]-dano,0)
        msg += "Jogador "+str(player)+" sofre "+str(dano)+" de dano e está com "+str(status[player]["vida"])+" de vida!\n"
  return (msg,at_list_log) # retorna mensagem e log

# E finalmente a função que vai rodar o jogo!

In [ ]:
# iniciais: lista de funções de configuração inicial
# turnos: lista de funções de turnos
# modo: 0 para indicar que é para imprimir mensagens.
# 1 para indicar que é para rodar sem imprimir

def jogo(iniciais, turnos, modo):
  log = dict() #para logar o que aconteceu
  log["status"]=dict()
  log["ataque"]=dict()

  n = len(iniciais) #quantidade de jogadores

  status = dict() 
  
  #iniciando os status dos jogadores
  (msg,cond) = calcula_status_inicial(status, iniciais)
  if modo==0:
      print(msg)
  if cond==False:
    return

  #lidando com colisoes
  msg = atualiza_posicoes(status, [])
  print(msg,end = '')

  # realizando turnos
  for i in range(0, max_turnos):
    # verificando se o jogo já acabou
    if acabou(status):
      break

    print("\n-----------\nTurno",(i+1))
    move_list = []
    at_list = []
    for player in status.keys():
      if status[player]["vida"]>0:
        move = turnos[player](status, player)
        if move in ["DF_INC", "SORTE_INC", 
           "FOGO", "AGUA", "PLANTA", "LUZ", "ESCURIDAO", "NORMAL"]:
          msg = atualiza_tipo_defesa_sorte(status, player, move)
          if modo==0:
            print(msg,end = '')
        elif move in ["N","S","L","O","NO","NL","SL","SO"]:
          move_list.append([player, move])
        elif move in ["AT","AE"]:
          at_list.append([player, move])
        else:
          if modo==0:
            print("Jogador "+str(player)+" escolheu movimento inválido "+str(move))

    #atualiza posicoes
    msg = atualiza_posicoes(status, move_list)
    if modo==0:
      print(msg,end = '')

    # realiza ataques
    msg,at_list_log = atualiza_ataques(status, at_list)
    if modo==0:
      print(msg,end = '')

    # guardando informações
    log["status"][i] = copy.deepcopy(status)
    log["ataque"][i] = copy.deepcopy(at_list_log)
  return log

Veja como rodar o jogo. 

Criamos uma lista com as funções de configuração iniciais e uma lista com as funções de turno (em ordem por jogador).

## JOGANDO! Você é o jogador 0 neste exemplo!

In [ ]:
def turno_t(status, jogador):
  j = 0
  op = 1
  dir = "DF_INC"
  if status[op]["vida"] > status[j]["vida"]:
    dir = ""
    if status[op]["y"] > status[j]["y"]:
      dir += "S"
    elif status[op]["y"] < status[j]["y"]:
      dir += "N"
    if status[op]["x"] > status[j]["x"]:
      dir += "O"
    elif status[op]["x"] < status[j]["x"]:
      dir += "L"
  elif prox(status,j,op,raio,mapa_x,mapa_y):
    return "AT"
  else:
    dir = ""
    if status[op]["y"] > status[j]["y"]:
      dir += "N"
    elif status[op]["y"] < status[j]["y"]:
      dir += "S"
    if status[op]["x"] > status[j]["x"]:
      dir += "L"
    elif status[op]["x"] < status[j]["x"]:
      dir += "O"
  return dir


In [ ]:
iniciais = [inicial, inicial_exemplo]
turnos = [turno, turno_aleatorio]
log = jogo(iniciais, turnos, 0)

In [ ]:
iniciais = [inicialArena, inicial_exemplo, inicial_exemplo, inicial_exemplo,inicial_exemplo]
turnos = [turnoArena, turno_aleatorio, turno_so_ataca, turno_so_ataca, turno_aleatorio]
log = jogo(iniciais, turnos, 0)

# Animação 

Se quiser ver uma animação do jogo, rode a célula abaixo após rodar o jogo.

As cores mostram o tipo do jogador:

* marrom = NORMAL

* azul = AGUA

* vermelho = FOGO

* verde = PLANTA

* amarelo = LUZ

* roxo = ESCURIDAO

* cinza = DEU RUIM

O quadrado mostra que um jogador está atacando e mostra seu raio de ataque.

In [ ]:
x = dict()
y = dict()
size = dict()
color = dict()
text = dict()
jogador = dict()
s = 100
marrom = "#855000"
azul = "#0021c4"
vermelho = "#FF4C2C"
verde = "#078a00"
amarelo = "#ffd000"
roxo = "#451f78"
cinza = "#B2B2B2"
m = 0
for i in range(0,max_turnos):
  x[i] = []
  y[i] = []
  size[i] = []
  color[i] = []
  text[i] = []
  jogador[i] = []
  if i in log["status"].keys():
    m = i
    ii = i
  else:
    ii = m
  for player in log["status"][ii].keys():
    p = log["status"][ii][player]
    if p["vida"]>0:
      x[i].append(p["x"])
      y[i].append(p["y"])
      size[i].append(s)
      text[i].append(str(player)+'\n'+str(p["vida"]))
      jogador[i].append(player)
      tipo = p["tipo"]
      if tipo=="NORMAL":
        color[i].append(marrom)
      elif tipo=="AGUA":
        color[i].append(azul)
      elif tipo=="PLANTA":
        color[i].append(verde)
      elif tipo=="FOGO":
        color[i].append(vermelho)
      elif tipo=="ESCURIDAO":
        color[i].append(roxo)
      elif tipo=="LUZ":
        color[i].append(amarelo)
      else:
        color[i].append(cinza)

camera = Camera(plt.figure())
plt.axis("equal")

for i in range(0,m):
    plt.scatter(x[i],y[i],c=color[i], s=size[i])
    for j in range(0,len(text[i])):
      plt.text(x[i][j],y[i][j],text[i][j])
      for [k, at, r, tipo] in log["ataque"][i]:
        if k==jogador[i][j]:
          c = cinza
          if at=="AT":
            c=marrom
          else:
            if tipo=="NORMAL":
              c = marrom
            elif tipo=="AGUA":
              c = azul
            elif tipo=="PLANTA":
              c=verde
            elif tipo=="FOGO":
              c=vermelho
            elif tipo=="ESCURIDAO":
              c= roxo
            elif tipo=="LUZ":
              c=amarelo
          plt.plot([x[i][j]-r, x[i][j]+r, x[i][j]+r, x[i][j]-r,x[i][j]-r],
                   [y[i][j]-r, y[i][j]-r, y[i][j]+r, y[i][j]+r,y[i][j]-r],c=c)
          for delta in range(0,21):
            xx = np.arange(x[i][j]-r, x[i][j]+r)
            plt.plot(xx, (y[i][j]+delta*2*r/20-r)*np.array([1]*len(xx)),c=c,alpha =0.8)   
    camera.snap()

from IPython.display import HTML
anim = camera.animate(blit=True,interval=1500,repeat=False)
HTML(anim.to_html5_video())